In [11]:
import pandas as pd
import plotly.express as px
from   datetime import date, timedelta, datetime
from dateutil import relativedelta
pd.set_option('display.max_columns', 500)
dt_string = datetime.now().strftime('%y%m%d')

In [2]:
tienda = ['101', '108', '93', '6', '183', '25', '38', '53', '13', '43', '82',
         '138', '72', '35', '60', '123', '98', '36', '19', '18', '50', '45',
         '85', '56', '96','322', '131', '139', '141', '143', '5', '142', '37']
cd = [ '9951', '9921', '3000', '9903', '2000', '2001', '2002', '9970', '9901', '3002', '9905', '9961']

In [6]:
f3_name = '220316-1015-f3-output'

mes_1= '2022-02-28' # calculo de 30 días atras

['22', 2, '16']

In [64]:
datetime.now() - timedelta(29)

datetime.datetime(2022, 2, 15, 11, 38, 41, 821209)

In [4]:
f3 = pd.read_csv(f'input/{f3_name }.csv', sep=';', dtype='object')

f3["tipo_producto"] = f3["tipo_producto"].str.title()

# Clasificar locales
f3.loc[f3.local.isin(tienda), 'local_agg'] = 'TIENDA'
f3.loc[f3.local.isin(cd), 'local_agg'] = 'CD'
f3.loc[f3.local_agg.isna(), 'local_agg'] = 'OTROS'

#Cantidades
f3['cant*costoprmd'] = pd.to_numeric(f3['cant*costoprmd'])

#Fechas
fechas = ['fecha_reserva', 'fecha_envio', 'fecha_anulacion','fecha_confirmacion']
f3.loc[:, fechas] = f3[fechas].apply(lambda x: x.replace(["ene", "abr", "ago", "dic"], ["jan", "apr", "aug", "dec"], regex=True))
for i in fechas:f3[i] = pd.to_datetime(f3[i], format='%d-%b-%Y')
f3['mes'] = f3["fecha_reserva"].dt.strftime('%b-%y')
f3 = f3.sort_values("fecha_reserva")

# Filtros
f3 = f3.loc[f3.fecha_reserva >='2021-01-01']
f3_abierto = f3.loc[(f3['descripcion6']=='enviado') | (f3 ['descripcion6']=='reservado')].reset_index()

# Groupbys
gb_f3_abierto = f3_abierto.groupby(['tipo_producto', 'mes'])['cant*costoprmd'].sum()

F3 ABIERTOS POR FECHA DE RESERVA


In [5]:
def set_columns_sum(base, var):    
    lista = base.loc[base[var].notna()][var].unique()
    gb_var = base.groupby(var)['cant*costoprmd'].sum().reset_index()
    gb_var.set_index(var, inplace=True)
    for item in lista:
        base.loc[base[var]==item, var] = f'{item} - ${round(gb_var.loc[item, "cant*costoprmd"]/1e6):,.0f}M'
    return base 

In [6]:
f3_promkp_abierto = f3_abierto.loc[(f3_abierto ['tipo_producto']=='Producto') | (f3_abierto ['tipo_producto']=='Market Place')].reset_index() #MKP y producto
f3_total_prd = f3_promkp_abierto.groupby(['tipo_producto'])['cant*costoprmd'].sum() #Costo mkp y producto
print("f3 abiertos" + str(f3_total_prd)+"654654654654654654654654654654654654654654")
print(f'---Costo total de F3 abiertos: ${f3_total_prd.sum()/1e6:,.0f} M')  #Costo total
print(f'---Costo total F3 abiertos (mkp y producto) millones: \n {f3_total_prd/1e6}')

f3_mayor_90= f3_promkp_abierto.loc[f3_promkp_abierto["fecha_reserva"] <= mes_1]
print('---Costo total F3 abiertos mayor a 90 días: \n', f3_mayor_90.groupby(['tipo_producto'])['cant*costoprmd'].sum()/1e6) #Costo mkp y producto > 90 días

gb_f3m90 = f3_mayor_90.groupby(['tipo_producto'])['cant*costoprmd'].sum().reset_index()
f3_total_3m = ('{:,.0f} M '.format(gb_f3m90['cant*costoprmd'].sum()/1e6))
f3_mkp_3m = ('{:,.0f} M '.format(gb_f3m90.loc[gb_f3m90.tipo_producto =='Market Place', 'cant*costoprmd'].sum()/1e6))
f3_prd_3m = ('{:,.0f} M '.format(gb_f3m90.loc[gb_f3m90.tipo_producto =='Producto', 'cant*costoprmd'].sum()/1e6))
# F3s abiertos según fecha de reserva
grupo_F3_prd_mkp = f3_promkp_abierto.groupby(["tipo_producto",'mes'], sort =False)['cant*costoprmd'].sum().reset_index()
grupo_F3_prd_mkp = set_columns_sum(grupo_F3_prd_mkp, "tipo_producto")
grupo_F3_prd_mkp = set_columns_sum(grupo_F3_prd_mkp, "mes")


f3 abiertostipo_producto
Market Place     717835967
Producto        2518586820
Name: cant*costoprmd, dtype: int64654654654654654654654654654654654654654654
---Costo total de F3 abiertos: $3,236 M
---Costo total F3 abiertos (mkp y producto) millones: 
 tipo_producto
Market Place     717.835967
Producto        2518.586820
Name: cant*costoprmd, dtype: float64
---Costo total F3 abiertos mayor a 90 días: 
 tipo_producto
Market Place     420.153470
Producto        2033.595612
Name: cant*costoprmd, dtype: float64


In [7]:
f3_mayor_90.groupby(['tipo_producto'])['cant*costoprmd'].sum()/1e6

tipo_producto
Market Place     420.153470
Producto        2033.595612
Name: cant*costoprmd, dtype: float64

In [8]:
f3_promkp_abierto.groupby(["tipo_producto",'mes'], sort =False)['cant*costoprmd'].sum()/1e6

tipo_producto  mes   
Market Place   Oct-21     24.280659
               Nov-21     28.826436
               Dec-21     55.374511
Producto       Dec-21    161.168620
               Jan-22    922.801617
Market Place   Jan-22    104.611405
Producto       Feb-22    949.625375
Market Place   Feb-22    207.060459
               Mar-22    297.682497
Producto       Mar-22    484.991208
Name: cant*costoprmd, dtype: float64

In [39]:
grafica_F3_prd_mkp=px.bar(grupo_F3_prd_mkp, x="mes", y="cant*costoprmd", labels={'mes':'Mes de reserva', "cant*costoprmd": "Costo promedio", 'tipo_producto':'Tipo producto'}, 
text='cant*costoprmd', text_auto='.2s', color = 'tipo_producto', color_discrete_sequence=['rgb(36, 121, 108)','rgb(204, 97, 176)'], title=f"F3 abiertos según fecha de reserva - Total costo ${round(f3_promkp_abierto['cant*costoprmd'].sum()/1e6):,.0f}M" )
grafica_F3_prd_mkp.update_layout(legend=dict(yanchor="top", y=0.98, xanchor="left", x=1))
 
grafica_F3_prd_mkp.add_shape(type="rect",
    xref="paper", yref="paper",
    x0=0, y0=0,
    x1=0.87, y1=1,
    line=dict(
        color="red",
        width=2,
    ))

grafica_F3_prd_mkp.add_annotation(x="Oct-21 - $24M", y=0.9*1e9,
        text= f"Total > 30 días= {f3_total_3m}",
        showarrow=False,
        font = dict (color = "red",size = 15))
grafica_F3_prd_mkp.add_annotation(x="Oct-21 - $24M", y=0.7*1e9,
        text= f"Market place > 30 días= {f3_mkp_3m} <br>Producto > 30 días= {f3_prd_3m}",
        showarrow=False,
        font = dict (color = "red",size = 12))
grafica_F3_prd_mkp.update_annotations(align="left")
grafica_F3_prd_mkp.write_image(f'images_f3/{dt_string}_f3_abiertos_fecha_reserva.svg' ,width=600, height=400)
grafica_F3_prd_mkp.show()

F3 MKP ABIERTO POR SEDE

In [40]:
f3_mkp_abierto = f3_abierto.loc[f3_abierto['tipo_producto']=='Market Place'].reset_index()
print('Fecha de F3 abierto más antiguo= ', f3_mkp_abierto.fecha_reserva.min().strftime(format='%d-%b-%Y'))
print(f'Costo total de F3 abiertos MKP= {f3_mkp_abierto["cant*costoprmd"].sum()/1e6:,.0f} M')
print('Costo de f3 abierto mkp por local_agg')
print(f3_mkp_abierto.groupby(['local_agg'])['cant*costoprmd'].sum()/1e6)

# F3s mkp abiertos por sede 
mkp_sede = f3_mkp_abierto.groupby(['local_agg','mes'], sort=False)['cant*costoprmd'].sum().reset_index()
mkp_sede = set_columns_sum(mkp_sede, 'local_agg')
mkp_sede = set_columns_sum(mkp_sede, 'mes')


Fecha de F3 abierto más antiguo=  01-Oct-2021
Costo total de F3 abiertos MKP= 718 M
Costo de f3 abierto mkp por local_agg
local_agg
CD        255.521275
OTROS      45.669107
TIENDA    416.645585
Name: cant*costoprmd, dtype: float64


In [41]:
co ={"mes": ["Jan-21", "Feb-21", "Mar-21", "Apr-21", "May-21", "Jun-21", "Jul-21", "Aug-21", "Sep-21", "Oct-21", "Nov-21", "Dec-21", 'Jan-22', 'Feb-22'], 'local_agg':['CD', 'TIENDA', 'OTROS']}
fig_mkp_sede = px.bar(mkp_sede, x ='mes', y='cant*costoprmd', color='local_agg', title=f"F3 Marketplace abiertos por sede - Total costo ${f3_mkp_abierto['cant*costoprmd'].sum()/1e6:,.0f}M",color_discrete_sequence=[ 'rgb(47, 138, 196)','rgb(153, 201, 69)','rgb(165, 170, 153)' ], labels={ 'cant*costoprmd':'Costo promedio', 'mes':'Mes de reserva', 'local_agg':'Local'}, text='cant*costoprmd',text_auto='.2s' ) #, category_orders=co) 
fig_mkp_sede.update_layout(legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.1))
fig_mkp_sede.write_image(f'images_f3/{dt_string}_f3_abierto_sede.svg',width=600, height=350) # 820 , 350 
fig_mkp_sede.show()

### Tendencias

In [47]:
calc_tend = f3_abierto.loc[(f3_abierto ['tipo_producto']=='Producto') | (f3_abierto ['tipo_producto']=='Market Place')].groupby(["tipo_producto", "mes"])["cant*costoprmd"].sum().reset_index()
calc_tend.to_excel(f'input/{dt_string}_calculo_tendencia_f3.xlsx',index=False)

In [48]:
f3_abierto.loc[(f3_abierto ['tipo_producto']=='Producto') | (f3_abierto ['tipo_producto']=='Market Place')].groupby(["tipo_producto", "mes"])["cant*costoprmd"].sum().reset_index()

,tipo_producto,mes,cant*costoprmd
0,Market Place,Dec-21,55374511
1,Market Place,Feb-22,207060459
2,Market Place,Jan-22,104611405
3,Market Place,Mar-22,297682497
4,Market Place,Nov-21,28826436
5,Market Place,Oct-21,24280659
6,Producto,Dec-21,161168620
7,Producto,Feb-22,949625375
8,Producto,Jan-22,922801617
9,Producto,Mar-22,484991208


In [49]:
fecha_inical = "2021-12-15"

In [52]:
f3_tendencia = pd.read_excel('input/tendencias.xlsx', dtype=str, sheet_name='f3')

# Producto
f3_tendencia['COSTO_REDON'] = pd.to_numeric(f3_tendencia['COSTO_REDON'])
f3_tendencia['FECHA_PPT'] = pd.to_datetime(f3_tendencia['FECHA_PPT'], format='%Y-%m-%d')

f3_tend_prd = f3_tendencia.loc[(f3_tendencia["TIPO_PRODUCTO"] == "Producto") & (f3_tendencia["FECHA_PPT"] > fecha_inical ) ]
f3_tend_grupo_p = f3_tend_prd.groupby(["TIPO_PRODUCTO", "FECHA_PPT"])["COSTO_REDON"].sum().reset_index()
f3_tend_grupo_p["COSTO_REDON"] = ((f3_tend_grupo_p["COSTO_REDON"]/1e6).round())

total_tend_prd = f3_tend_grupo_p["COSTO_REDON"].sum()

# Marketplace
f3_tend_mkp = f3_tendencia.loc[(f3_tendencia["TIPO_PRODUCTO"] == "Market Place") & (f3_tendencia["FECHA_PPT"] > fecha_inical) ]
f3_tend_grupo_mkp=f3_tend_mkp.groupby(["TIPO_PRODUCTO", "FECHA_PPT"])["COSTO_REDON"].sum().reset_index()
f3_tend_grupo_mkp["COSTO_REDON"] = ((f3_tend_grupo_mkp["COSTO_REDON"]/1e6).round())
total_tend_mkp = f3_tend_grupo_mkp["COSTO_REDON"].sum()

In [53]:
f3_tend_grupo_p

,TIPO_PRODUCTO,FECHA_PPT,COSTO_REDON
0,Producto,2021-12-20,4199.0
1,Producto,2021-12-31,1003.0
2,Producto,2022-01-17,719.0
3,Producto,2022-01-31,161.0
4,Producto,2022-02-23,1893.0
5,Producto,2022-03-01,2037.0
6,Producto,2022-03-16,2519.0


In [54]:
grafica_tendencia_f3 = px.line(f3_tend_grupo_p, x="FECHA_PPT", y="COSTO_REDON", labels={'FECHA_PPT':'Fecha de corte', "COSTO_REDON": "Costo promedio (Millones)" },
text='COSTO_REDON', color_discrete_sequence=['rgb(204, 97, 176)'], title=f"Tendencia F3 abierto tipo producto según fecha de corte" )
grafica_tendencia_f3.update_layout(legend=dict(yanchor="top", y=1, xanchor="left", x=0.45))
grafica_tendencia_f3.update_traces(textposition="bottom center")
grafica_tendencia_f3.update_xaxes(range=['2021-15-15','2022-25-10'], constrain="domain")
grafica_tendencia_f3.write_image(f"images_f3/{dt_string}_f3_TENDENCIA_Producto.svg",width=650, height=400)
grafica_tendencia_f3.show()

In [55]:
grafica_tend_f3_mp = px.line(f3_tend_grupo_mkp, x="FECHA_PPT", y="COSTO_REDON", labels={'FECHA_PPT':'Fecha de corte', "COSTO_REDON": "Costo promedio (Millones)" },
text='COSTO_REDON', color_discrete_sequence=['rgb(36, 121, 108)','rgb(204, 97, 176)'], title=f"Tendencia F3 abierto tipo marketplace según fecha de corte" )
grafica_tend_f3_mp.update_layout(legend=dict(yanchor="top", y=1, xanchor="left", x=0.48))
grafica_tend_f3_mp.update_traces(textposition="bottom right")
grafica_tend_f3_mp.update_xaxes(range=['2021-12-15','2022-25-10'], constrain="domain")
grafica_tend_f3_mp.write_image(f"images_f3/{dt_string}_f3_TENDENCIA_MKP.svg",width=650, height=400)
grafica_tend_f3_mp.show()